In [23]:
import os

# Download the dataset from Kaggle
os.system('kaggle datasets download -d mmmarchetti/tweets-dataset')
os.system('tar -xvf tweets-dataset.zip') # Unzip the dataset using tar - windows



0

In [14]:
import pandas as pd

In [15]:
# Load the CSV file into a pandas DataFrame

df = pd.read_csv('tweets.csv')

In [16]:
df.head()

,author,content,country,date_time,id,language,latitude,longitude,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,NaN,12/01/2017 19:52,8.196330e+17,en,NaN,NaN,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,NaN,11/01/2017 08:38,8.191010e+17,en,NaN,NaN,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,NaN,11/01/2017 02:52,8.190140e+17,en,NaN,NaN,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,NaN,11/01/2017 02:44,8.190120e+17,en,NaN,NaN,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,NaN,10/01/2017 05:22,8.186890e+17,en,NaN,NaN,17620,4655


In [17]:
# preprocess to include date, message, location
df.country.value_counts()

country
Vereinigte Staaten    25
Portugal              11
Name: count, dtype: int64

In [18]:
# drop those with country NA
df = df.dropna(subset=['country'])
df.head()

,author,content,country,date_time,id,language,latitude,longitude,number_of_likes,number_of_shares
12993,YouTube,Feast your eyes and ears. Check out @deadmau5’...,Vereinigte Staaten,16/12/2016 23:56,8.099100e+17,en,NaN,NaN,1049,172
13152,YouTube,Congrats to @BJTHECHICAGOKID on 3 #GRAMMYs nom...,Vereinigte Staaten,07/12/2016 03:24,8.063390e+17,en,NaN,NaN,601,95
13153,YouTube,No features on this track. 👀\r\n\r\nNew @JCole...,Vereinigte Staaten,07/12/2016 02:08,8.063200e+17,en,NaN,NaN,1204,324
13446,YouTube,@coolcat1911 @coolcat1911 2 scary 4 us,Vereinigte Staaten,31/10/2016 20:42,7.931910e+17,en,NaN,NaN,15,6
13447,YouTube,"@sebasdelvaux @sebasdelvaux Hey, hey! Thanks f...",Vereinigte Staaten,31/10/2016 20:42,7.931910e+17,en,NaN,NaN,19,8


In [19]:
df.drop(['author','id','language','longitude','latitude','number_of_likes','number_of_shares'], axis=1, inplace=True)
df.head()

,content,country,date_time
12993,Feast your eyes and ears. Check out @deadmau5’...,Vereinigte Staaten,16/12/2016 23:56
13152,Congrats to @BJTHECHICAGOKID on 3 #GRAMMYs nom...,Vereinigte Staaten,07/12/2016 03:24
13153,No features on this track. 👀\r\n\r\nNew @JCole...,Vereinigte Staaten,07/12/2016 02:08
13446,@coolcat1911 @coolcat1911 2 scary 4 us,Vereinigte Staaten,31/10/2016 20:42
13447,"@sebasdelvaux @sebasdelvaux Hey, hey! Thanks f...",Vereinigte Staaten,31/10/2016 20:42


In [20]:
# preprocess date_time to include year only
df['date_time'] = pd.to_datetime(df['date_time']).dt.year
df.head()

C:\Users\ammar\AppData\Local\Temp\ipykernel_31972\232267393.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date_time'] = pd.to_datetime(df['date_time']).dt.year


,content,country,date_time
12993,Feast your eyes and ears. Check out @deadmau5’...,Vereinigte Staaten,2016
13152,Congrats to @BJTHECHICAGOKID on 3 #GRAMMYs nom...,Vereinigte Staaten,2016
13153,No features on this track. 👀\r\n\r\nNew @JCole...,Vereinigte Staaten,2016
13446,@coolcat1911 @coolcat1911 2 scary 4 us,Vereinigte Staaten,2016
13447,"@sebasdelvaux @sebasdelvaux Hey, hey! Thanks f...",Vereinigte Staaten,2016


In [21]:
# Select the columns you need and write them to a text file
df.rename(columns={'date_time': 'date', 'content': 'message', 'country': 'location'}, inplace=True)

df[['date', 'message', 'location']].to_csv('twitter_data.txt', index=False, header=False)


In [2]:
%%file task1.py
# in this file i have ran task1 as well as setup MRjob

# import pip

# pip upgrade pip
# pip.main(["install", "--upgrade", "pip"])
# pip.main(["install", "MRjob"])
# pip.main(['install', 'kaggle'])

import mrjob
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+")

class MRWordFrequencyCount(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_total_wordcount)
        ]

    def mapper_get_words(self, _, line):
        # split the line into words
        date, message, location = line.split(',', 2)
        words = WORD_RE.findall(message)
        for word in words:
            yield (date, word.lower()), 1

    def reducer_count_words(self, word, counts):
        yield word, sum(counts)

    def reducer_find_total_wordcount(self, word, counts):
        yield word[0], (word[1], sum(counts))

if __name__ == '__main__':
    MRWordFrequencyCount.run()
# task will be run as: 'python task1.py twitter_data.txt > task1.txt'

Writing task1.py


In [3]:
! python task1.py twitter_data.txt > task1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ammar\AppData\Local\Temp\task1.ammar.20240220.215845.237960
Running step 1 of 2...
Running step 2 of 2...
job output is in C:\Users\ammar\AppData\Local\Temp\task1.ammar.20240220.215845.237960\output
Streaming final output from C:\Users\ammar\AppData\Local\Temp\task1.ammar.20240220.215845.237960\output...
Removing temp directory C:\Users\ammar\AppData\Local\Temp\task1.ammar.20240220.215845.237960...


In [4]:
%%file task2.py
from mrjob.job import MRJob

class MRWordCount(MRJob):

    def mapper(self, _, line):
        year, word_count = line.split('\t')
        word, count = eval(word_count)
        yield word, count

    def reducer(self, word, counts):
        yield word, sum(counts)

if __name__ == '__main__':
    MRWordCount.run()

# task will be run as: 'python task2.py task1.txt > task2.txt'

Writing task2.py


In [5]:
! python task2.py task1.txt > task2.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\ammar\AppData\Local\Temp\task2.ammar.20240220.215849.312081
Running step 1 of 1...
job output is in C:\Users\ammar\AppData\Local\Temp\task2.ammar.20240220.215849.312081\output
Streaming final output from C:\Users\ammar\AppData\Local\Temp\task2.ammar.20240220.215849.312081\output...
Removing temp directory C:\Users\ammar\AppData\Local\Temp\task2.ammar.20240220.215849.312081...
